In [1]:
cd data

/home/ubuntu/PRNU/data


In [2]:
import tensorflow as tf

from keras.models import Model
from keras.engine import InputSpec
from keras.engine.topology import Layer
from keras.layers import Input, Conv2D, Activation, BatchNormalization
from keras.layers.merge import Add
from keras.utils import conv_utils
from keras.layers.core import Dropout


def res_block(input, filters, kernel_size=(3, 3), strides=(1, 1), use_dropout=False):
    """
    Instanciate a Keras Resnet Block using sequential API.
    :param input: Input tensor
    :param filters: Number of filters to use
    :param kernel_size: Shape of the kernel for the convolution
    :param strides: Shape of the strides for the convolution
    :param use_dropout: Boolean value to determine the use of dropout
    :return: Keras Model
    """
    x = ReflectionPadding2D((1, 1))(input)
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if use_dropout:
        x = Dropout(0.5)(x)

    x = ReflectionPadding2D((1, 1))(x)
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,)(x)
    x = BatchNormalization()(x)

    merged = Add()([input, x])
    return merged


def spatial_reflection_2d_padding(x, padding=((1, 1), (1, 1)), data_format=None):
    """
    Pad the 2nd and 3rd dimensions of a 4D tensor.
    :param x: Input tensor
    :param padding: Shape of padding to use
    :param data_format: Tensorflow vs Theano convention ('channels_last', 'channels_first')
    :return: Tensorflow tensor
    """
    assert len(padding) == 2
    assert len(padding[0]) == 2
    assert len(padding[1]) == 2
    if data_format is None:
        data_format = image_data_format()
    if data_format not in {'channels_first', 'channels_last'}:
        raise ValueError('Unknown data_format ' + str(data_format))

    if data_format == 'channels_first':
        pattern = [[0, 0],
                   [0, 0],
                   list(padding[0]),
                   list(padding[1])]
    else:
        pattern = [[0, 0],
                   list(padding[0]), list(padding[1]),
                   [0, 0]]
    return tf.pad(x, pattern, "REFLECT")


# TODO: Credits
class ReflectionPadding2D(Layer):
    """Reflection-padding layer for 2D input (e.g. picture).
    This layer can add rows and columns or zeros
    at the top, bottom, left and right side of an image tensor.
    # Arguments
        padding: int, or tuple of 2 ints, or tuple of 2 tuples of 2 ints.
            - If int: the same symmetric padding
                is applied to width and height.
            - If tuple of 2 ints:
                interpreted as two different
                symmetric padding values for height and width:
                `(symmetric_height_pad, symmetric_width_pad)`.
            - If tuple of 2 tuples of 2 ints:
                interpreted as
                `((top_pad, bottom_pad), (left_pad, right_pad))`
        data_format: A string,
            one of `channels_last` (default) or `channels_first`.
            The ordering of the dimensions in the inputs.
            `channels_last` corresponds to inputs with shape
            `(batch, height, width, channels)` while `channels_first`
            corresponds to inputs with shape
            `(batch, channels, height, width)`.
            It defaults to the `image_data_format` value found in your
            Keras config file at `~/.keras/keras.json`.
            If you never set it, then it will be "channels_last".
    # Input shape
        4D tensor with shape:
        - If `data_format` is `"channels_last"`:
            `(batch, rows, cols, channels)`
        - If `data_format` is `"channels_first"`:
            `(batch, channels, rows, cols)`
    # Output shape
        4D tensor with shape:
        - If `data_format` is `"channels_last"`:
            `(batch, padded_rows, padded_cols, channels)`
        - If `data_format` is `"channels_first"`:
            `(batch, channels, padded_rows, padded_cols)`
    """

    def __init__(self,
                 padding=(1, 1),
                 data_format=None,
                 **kwargs):
        super(ReflectionPadding2D, self).__init__(**kwargs)
        self.data_format = conv_utils.normalize_data_format(data_format)
        if isinstance(padding, int):
            self.padding = ((padding, padding), (padding, padding))
        elif hasattr(padding, '__len__'):
            if len(padding) != 2:
                raise ValueError('`padding` should have two elements. '
                                 'Found: ' + str(padding))
            height_padding = conv_utils.normalize_tuple(padding[0], 2,
                                                        '1st entry of padding')
            width_padding = conv_utils.normalize_tuple(padding[1], 2,
                                                       '2nd entry of padding')
            self.padding = (height_padding, width_padding)
        else:
            raise ValueError('`padding` should be either an int, '
                             'a tuple of 2 ints '
                             '(symmetric_height_pad, symmetric_width_pad), '
                             'or a tuple of 2 tuples of 2 ints '
                             '((top_pad, bottom_pad), (left_pad, right_pad)). '
                             'Found: ' + str(padding))
        self.input_spec = InputSpec(ndim=4)

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_first':
            if input_shape[2] is not None:
                rows = input_shape[2] + self.padding[0][0] + self.padding[0][1]
            else:
                rows = None
            if input_shape[3] is not None:
                cols = input_shape[3] + self.padding[1][0] + self.padding[1][1]
            else:
                cols = None
            return (input_shape[0],
                    input_shape[1],
                    rows,
                    cols)
        elif self.data_format == 'channels_last':
            if input_shape[1] is not None:
                rows = input_shape[1] + self.padding[0][0] + self.padding[0][1]
            else:
                rows = None
            if input_shape[2] is not None:
                cols = input_shape[2] + self.padding[1][0] + self.padding[1][1]
            else:
                cols = None
            return (input_shape[0],
                    rows,
                    cols,
                    input_shape[3])

    def call(self, inputs):
        return spatial_reflection_2d_padding(inputs,
                                             padding=self.padding,
                                             data_format=self.data_format)

    def get_config(self):
        config = {'padding': self.padding,
                  'data_format': self.data_format}
        base_config = super(ReflectionPadding2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

Using TensorFlow backend.


In [3]:
from keras.layers import Input, Activation, Add
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.core import Lambda
from keras.layers.normalization import BatchNormalization
from keras.models import Model

ngf = 64
input_nc = 3
output_nc = 3
input_shape_generator = (256, 256, input_nc)
n_blocks_gen = 9


def generator_model():
    """Build generator architecture."""
    # Current version : ResNet block
    inputs = Input(shape=image_shape)

    x = ReflectionPadding2D((3, 3))(inputs)
    x = Conv2D(filters=ngf, kernel_size=(7,7), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Increase filter number
    n_downsampling = 2
    for i in range(n_downsampling):
        mult = 2**i
        x = Conv2D(filters=ngf*mult*2, kernel_size=(3,3), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    # Apply 9 ResNet blocks
    mult = 2**n_downsampling
    for i in range(n_blocks_gen):
        x = res_block(x, ngf*mult, use_dropout=True)

    # Decrease filter number to 3 (RGB)
    for i in range(n_downsampling):
        mult = 2**(n_downsampling - i)
        x = Conv2DTranspose(filters=int(ngf * mult / 2), kernel_size=(3,3), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    x = ReflectionPadding2D((3,3))(x)
    x = Conv2D(filters=output_nc, kernel_size=(7,7), padding='valid')(x)
    x = Activation('tanh')(x)

    # Add direct connection from input to output and recenter to [-1, 1]
    outputs = Add()([x, inputs])
    outputs = Lambda(lambda z: z/2)(outputs)

    model = Model(inputs=inputs, outputs=outputs, name='Generator')
    return model

In [4]:
from keras.layers import Input
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D
from keras.layers.core import Dense, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Model

ndf = 64
output_nc = 3
input_shape_discriminator = (256, 256, output_nc)


def discriminator_model():
    """Build discriminator architecture."""
    n_layers, use_sigmoid = 3, False
    inputs = Input(shape=input_shape_discriminator)

    x = Conv2D(filters=ndf, kernel_size=(4,4), strides=2, padding='same')(inputs)
    x = LeakyReLU(0.2)(x)

    nf_mult, nf_mult_prev = 1, 1
    for n in range(n_layers):
        nf_mult_prev, nf_mult = nf_mult, min(2**n, 8)
        x = Conv2D(filters=ndf*nf_mult, kernel_size=(4,4), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)

    nf_mult_prev, nf_mult = nf_mult, min(2**n_layers, 8)
    x = Conv2D(filters=ndf*nf_mult, kernel_size=(4,4), strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(filters=1, kernel_size=(4,4), strides=1, padding='same')(x)
    if use_sigmoid:
        x = Activation('sigmoid')(x)

    x = Flatten()(x)
    x = Dense(1024, activation='tanh')(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=x, name='Discriminator')
    return model

In [5]:
from keras.layers import Input
from keras.models import Model

def generator_containing_discriminator_multiple_outputs(generator, discriminator):
    inputs = Input(shape=image_shape)
    generated_images = generator(inputs)
    outputs = discriminator(generated_images)
    model = Model(inputs=inputs, outputs=[generated_images, outputs])
    return model

In [6]:
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model

image_shape = (256, 256, 3)

def perceptual_loss(y_true, y_pred):
    vgg = VGG16(include_top=False, weights='imagenet', input_shape=image_shape)
    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    loss_model.trainable = False
    return K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))

In [7]:
import keras.backend as K

def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true*y_pred)

In [8]:
cd /home/ubuntu/PRNU/data/imgs

/home/ubuntu/PRNU/data/imgs


In [9]:
import cv2
import numpy as np
import multiprocessing
import math
import rwt
from scipy import special
import csv
from random import *
import requests
import shutil
import sqlite3
import os
import imutils
import io
import datetime

In [10]:
def adapt_array(arr):
    # SQL util function - convert np array to sql array
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    # SQL util function - convert sql array to np array
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

def getNumber(s):
    no = ''
    i = -1
    while str.isdigit(s[i]):
        no += s[i]
        i -= 1
    no = no[::-1]
    return int(no)

def getKeys():
    # returns all user ids in the DB
    cur.execute('SELECT id FROM users')
    table_keys_t = cur.fetchall()
    table_keys = [i for i in range(len(table_keys_t))]
    for i in range(len(table_keys_t)):
        table_keys[i] = table_keys_t[i][0]
    del table_keys_t
    return table_keys

def mean2(x):
    y = np.sum(x) / np.size(x);
    return y

def corr2(a,b):
    a = a - mean2(a)
    b = b - mean2(b)

    r = (a*b).sum() / math.sqrt((a*a).sum() * (b*b).sum());
    return r

def threshold(y,t):
    res = y-t
    return (res+abs(res))/2

def removeNeighborhood(X,x,ssize):
    M,N = X.shape[0],X.shape[1]
    radius = int((ssize-1)/2)
    X = np.roll(X,radius-x[1]+1,axis=1)
    X = np.roll(X,radius-x[0]+1,axis=0)
    Y = X[ssize:,:ssize]
    Y = Y.flatten()
    Y = np.append(Y,X[M*ssize:].T)
    return Y

def qFunction(x):
    if(x<37.5):
        Q = 1/2*special.erfc(x/math.sqrt(2))
        logQ = math.log(Q)
    else:
        Q = (1/math.sqrt(2*math.pi))/np.multiply(x,np.exp(-np.divide(np.square(x),2)))
        logQ = -np.square(x)/2 - np.log(x) - 1/2*math.log(2*math.pi)
    return Q,logQ

def FAFromPCE(pce,search_space):
    p,logP = qFunction(np.sign(pce)*np.sqrt(np.abs(pce)))
    if(pce<50):
        FA = np.power(1-(1-p),search_space)
    else:
        FA = search_space * p
    if(FA==0):
        FA = search_space*p
        log10FA = np.log10(search_space) + logP*math.log10(2)
    else:
        log10FA = np.log10(FA)
    return FA,log10FA

def zeroMean(X):
    reshaped = 0
    if(len(X.shape)==2):
        X = np.reshape(X,[X.shape[0],X.shape[1],1])
        reshaped = 1
    M,N,K = X.shape[0],X.shape[1],X.shape[2]
    Y = np.zeros(X.shape)
    row = np.zeros([M,K])
    col = np.zeros([K,N])
    for i in range(K):
        t = np.mean(X[:,:,i])
        X[:,:,i] = X[:,:,i] - t
    for i in range(K):
        col[i,:] = np.mean(X[:,:,i],axis=0)
        row[:,i] = np.mean(X[:,:,i].T,axis=0).T
    for j in range(K):
        Y[:,:,j]=X[:,:,j]-np.ones([M,1])*col[j,:]
    for j in range(K):
        t = row[:,j]
        t.shape = [t.shape[0],1]
        Y[:,:,j]=Y[:,:,j]-t*np.ones([1,N])
    if reshaped is 1:
        Y = np.reshape(Y,[Y.shape[0],Y.shape[1]])
    return Y

def zeroMeanTotal(X):
    if len(X.shape) != 2:
        Y = np.zeros(X.shape)
        Z = zeroMean(X[::2,::2,:])
        Y[::2,::2,:] = Z
        Z = zeroMean(X[::2,1::2,:])
        Y[::2,1::2,:] = Z
        Z = zeroMean(X[1::2,::2,:])
        Y[1::2,::2,:] = Z
        Z = zeroMean(X[1::2,1::2,:])
        Y[1::2,1::2,:] = Z
    else:
        Y = np.zeros(X.shape)
        Z = zeroMean(X[::2,::2])
        Y[::2,::2] = Z
        Z = zeroMean(X[::2,1::2])
        Y[::2,1::2] = Z
        Z = zeroMean(X[1::2,::2])
        Y[1::2,::2] = Z
        Z = zeroMean(X[1::2,1::2])
        Y[1::2,1::2] = Z
    return Y

def waveNoise(coef,noiseVar):
    t = np.square(coef)
    filter = np.ones([3,3])/9
    coefVar = threshold(cv2.filter2D(t,-1,filter,borderType=cv2.BORDER_CONSTANT),noiseVar)
    for w in range(5,10,2):
        filter = np.ones([w,w])/(w*w)
        EstVar = threshold(cv2.filter2D(t,-1,filter,borderType=cv2.BORDER_CONSTANT),noiseVar)
        coefVar = np.minimum(coefVar, EstVar)
    return np.divide(np.multiply(coef,noiseVar),np.add(coefVar,noiseVar))

def noiseExtract(img,qmf,sigma,L):
    M,N = img.shape[0],img.shape[1]
    m = 2**L
    minpad = 2
    nr = math.ceil((M+minpad)/m)*m
    nc = math.ceil((N+minpad)/m)*m
    pr = math.ceil((nr-M)/2)
    prd= math.floor((nr-M)/2)
    pc = math.ceil((nc-N)/2)
    pcr= math.floor((nc-N)/2)
    t1 = np.insert(np.insert(img[pr-1::-1,pc-1::-1],pc,img[pr-1::-1,:].T,axis=1),N+pc,img[pr-1::-1,N-1:N-pcr-1:-1].T,axis=1)
    t2 = np.insert(np.insert(img[:,pc-1::-1],pc,img.T,axis=1),N+pc,img[:,N-1:N-pcr-1:-1].T,axis=1)
    t3 = np.insert(np.insert(img[M-1:M-prd-1:-1,pc-1::-1],pc,img[M-1:M-prd-1:-1,:].T,axis=1),N+pc,img[M-1:M-prd-1:-1,N-1:N-pcr-1:-1].T,axis=1)
    img = np.insert(np.insert(t2,[pr-1],t1,axis=0),pr-1+M,t3,axis=0)
    # img = np.float32([[img[pr:1:-1,pc:1:-1],img[pr:1:-1,:],img[pr:1:-1,N:N-pcr+1:-1]],[img[:,pc:1:-1],img,img[:,N:N-pcr+1:-1]],[img[M:M-prd+1:-1,pc:1:-1],img[M:M-prd+1:-1,:],img[M:M-prd+1:-1,N:N-pcr+1:-1]]])    
    img = np.float64(img)
    NoiseVar = sigma**2
    wave_trans = np.zeros([nr,nc])
    # print(img.shape)
    # cA,(cH,cV,cD) = pywt.dwt2(img,'db4','per')
    # wave_trans = np.append(np.append(cA,cH,0),np.append(cV,cD,0),1)
    wave_trans = rwt.dwt(img,qmf,L)[0]
    for i in range(L):
        # Hhigh = [k for k in range(int(nc/2)+1,nc+1)]
        # Hlow = [k for k in range(1,int(nc/2)+1)]
        # Vhigh = [k for k in range(int(nr/2)+1,nr+1)]
        # Vlow = [k for k in range(1,int(nr/2)+1)]
        wave_trans[0:int(nr/2),int(nc/2):nc] = np.around(waveNoise(wave_trans[0:int(nr/2),int(nc/2):nc],NoiseVar),4)
        wave_trans[int(nr/2):nr,0:int(nc/2)] =  np.around(waveNoise(wave_trans[int(nr/2):nr,0:int(nc/2)],NoiseVar),4)
        wave_trans[int(nr/2):nr,int(nc/2):nc] = np.around(waveNoise(wave_trans[int(nr/2):nr,int(nc/2):nc],NoiseVar),4)
        nc = int(nc/2)
        nr = int(nr/2)
    wave_trans[:nr,:nc] = 0
    # cA = wave_trans[:136,:136]
    # cV = wave_trans[:136,136:]
    # cH = wave_trans[136:,:136]
    # cD = wave_trans[136:,136:]
    # image_noise = pywt.idwt2((cA,(cH,cV,cD)),'db4','per')
    image_noise = np.around(rwt.idwt(wave_trans,qmf,L)[0],4)
    return image_noise[pr:pr+M,pc:pc+N]

def noiseExtractFromImg(img,sigma):
    L = 4
    qmf = np.array([0.2304,0.7148,0.6309,-0.0280,-0.1870,0.0308,0.0329,-0.0106],dtype=np.float64)
    noise = np.zeros(img.shape)
    for j in range(3):
        noise[:,:,j] = noiseExtract(img[:,:,j],qmf,sigma,L)
    noise = noise.astype(np.float32)
    noise = zeroMeanTotal(noise)
    return noise

def intenScale(c):
    T = 252
    v = 6
    out = np.exp(-1*np.divide(np.square(np.subtract(c,T)),v))
    out[c<T] = np.divide(c[c<T],T)
    return np.around(out,4)

def getSaturMap(X):
    X = X.astype(np.uint8)
    M,N = X.shape[0],X.shape[1]
    if(X.max()<=250):
        return np.ones(X.shape)
    Xh = X - np.roll(X,1,axis=1)
    Xv = X - np.roll(X,1,axis=0)
    saturMap = np.bitwise_and(Xh,np.bitwise_and(Xv,np.bitwise_and(np.roll(Xh,-1,axis=1),np.roll(Xv,-1,axis=0))))
    if(len(X.shape)==3):
        for i in range(3):
            maxI[i] = X[:,:,i].max()
            if maxI[i] > 250:
                saturMap[:,:,j] = np.bitwise_not(np.bitwise_and(np.uint8((X[:,:,i]==maxI[i])*1),np.bitwise_not(saturMap[:,:,i])))
    else:
        maxX = X.max()
        saturMap = np.bitwise_not(np.bitwise_and(np.uint8((X==maxX)*1),np.bitwise_not(saturMap)))

    return (saturMap>=255)*1

def wienerFilter(noise,sigma):
    F = np.fft.fft2(noise)
    Fmag = np.abs(np.divide(F,math.sqrt(noise.shape[0]*noise.shape[1])))
    noiseVar = sigma**2
    Fmag1 = waveNoise(Fmag,noiseVar)
    Fmag[np.where(Fmag==0)] = 1
    Fmag1[np.where(Fmag==0)] = 0
    F = np.multiply(F,np.divide(Fmag1,Fmag))
    return np.around(np.real(np.fft.ifft2(F)),4)

def getFingerprint(imgs):
    sigma = 3
    L = 4
    qmf = np.array([0.2304,0.7148,0.6309,-0.0280,-0.1870,0.0308,0.0329,-0.0106],dtype=np.float64)
    RPsum = np.zeros([imgs.shape[3],imgs.shape[1],imgs.shape[2]])
    RP = np.zeros([imgs.shape[3],imgs.shape[1],imgs.shape[2]])
    NN = np.zeros([imgs.shape[3],imgs.shape[1],imgs.shape[2]])
    for i in imgs:
        i = i.astype(np.float32,copy=False)
        for j in range(3):
            imNoise = noiseExtract(i[:,:,j],qmf,sigma,L)
            inten = np.multiply(intenScale(i[:,:,j]),getSaturMap(i[:,:,j]))
            RPsum[j] = RPsum[j] + np.multiply(imNoise,inten)
            NN[j] = NN[j] + np.square(inten)
    
    for j in range(3):
        RP[j] = np.divide(RPsum[j],NN[j]+1)
    RP = cv2.merge(RP)
    RP = zeroMeanTotal(RP)
    RP = RP.astype(np.float32)
    return np.around(RP,4)

def getCorr(X,Y):
    X = np.subtract(X,np.mean(X))
    Y = np.subtract(Y,np.mean(Y))
    tiltedY = np.fliplr(Y)
    tiltedY = np.flipud(tiltedY)
    TA = np.fft.fft2(tiltedY)
    FA = np.fft.fft2(X)
    FF = np.multiply(TA,FA)
    return np.real(np.fft.ifft2(FF))

def getPCE(C):
    squaresize = 11
    shift_range = np.uint8([0,0])
    if np.any(shift_range>=C.shape):
        shift_range = np.minimum(shift_range,C.shape-1)
    cInRange = C[C.shape[0]-shift_range[0]-1:,C.shape[1]-shift_range[1]-1:]
    (ypeak,xpeak) = np.unravel_index(np.argmax(cInRange, axis=None), cInRange.shape)
    peakHeight = cInRange[ypeak,xpeak]
    peakLocation = shift_range+np.uint8([1,1])-np.uint8([ypeak,xpeak])
    cWithoutPeak = removeNeighborhood(C,np.uint8([ypeak,xpeak]),squaresize)
    correl = C[C.shape[0]-1,C.shape[1]-1]
    pceEnergy = np.mean(np.multiply(cWithoutPeak,cWithoutPeak))
    PCE = peakHeight**2/pceEnergy*np.sign(peakHeight)
    pValue = 1/2*special.erfc(peakHeight/math.sqrt(pceEnergy)/math.sqrt(2))
    pFA,log10pFA = FAFromPCE(PCE,(shift_range[0]+1)*(shift_range[1]+1))
    return pValue,PCE

def getResults(fingerprint1,img):
    test_img = img
    noisex = noiseExtractFromImg(test_img,2)
    noisex = wienerFilter(noisex,np.std(noisex))
    fingerprint1 = fingerprint1.astype(np.float32)
    # fingerprint2 = fingerprint2.astype(np.float32)
    noisex = noisex.astype(np.float32)
    img = img.astype(np.float32,copy=False)
    Ix = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    C = getCorr(noisex,np.multiply(Ix,fingerprint1))
    # C = getCorr(fingerprint2,fingerprint1)
    x,y = getPCE(C)
    return 1.0-x,y

def getFingerprintUtil(imgs):
    RP = getFingerprint(imgs)
    sigmaRP = np.std(RP)
    return wienerFilter(RP,sigmaRP)
    # if j<=1:
    #     RP = getFingerprint(imgs[:1,...])
    #     sigmaRP = np.std(RP)
    #     return wienerFilter(RP,sigmaRP)
    # elif j<=3:
    #     RP = getFingerprint(imgs[:2,...])
    #     sigmaRP = np.std(RP)
    #     return wienerFilter(RP,sigmaRP)
    # else:
    #     RP = getFingerprint(imgs[:3,...])
    #     sigmaRP = np.std(RP)
    #     return wienerFilter(RP,sigmaRP)

def checkDB(idc,img,sensor_res):
    keys = getKeys()
    print(keys)
    for i in range(len(keys)):
        cur.execute('SELECT fingerprint,width,height,pce1,pce2 from users where id=?',(keys[i],))
        tt = cur.fetchone()
        fp = tt[0]
        width = tt[1]
        height = tt[2]
        pce_val1 = tt[3]
        pce_val2 = tt[4]
        # print(img.shape)
        # print(str(width)+' , '+str(height))
        if(sensor_res[0]!=width or sensor_res[1]!=height):
            continue
        fp = fp.astype(np.float32)
        pce = getKorus(fp,img[0])
        pce2 = getKorus(fp,img[1])
        print('Attacker: '+str(idc)+' Defender: '+str(keys[i])+' PCE1: '+str(pce)+' PCE2: '+str(pce2)+' Req. PCE1: '+str(pce_val1)+' PCE2: '+str(pce_val2))
        max_val = 0
        if pce_val2<0.002 and pce_val2>0.001 and pce<0.002 and pce>0.001 and pce2<0.002 and pce2>0.001:
            print('<<<Duplicate - '+str(idc)+' and '+str(keys[i])+'>>>')
            writer.writerows([[idc,keys[i]]])
            continue
        if pce_val1<0.002 or pce_val2<0.002 or pce<0.002 or pce2<0.002:
            continue
        # if pce_val1>pce_val2:
        #     max_val = pce_val1
        #     min_val = pce_val2
        # else:
        #     max_val = pce_val2
        #     min_val = pce_val1
        max_val = pce_val2
        max_val = round(max_val,4)
        a = str(max_val)
        ind = a.rfind('0')
        if ind<=3:
            t = '0.0000'
            t = float(t[:ind+1]+'1')
            if pce>max_val-t and pce<max_val+t and pce2>max_val-t and pce2<max_val+t:
                print('<<<Duplicate - '+str(idc)+' and '+str(keys[i])+'>>>')
                writer.writerows([[idc,keys[i]]])

def getNearestNeighbour(idc,keys):
    max = 0.0
    cur.execute('SELECT fingerprint from users where id=?',(idc,))
    fingerPrint1 = cur.fetchone()[0]
    key = '0'
    for i in range(len(keys)):
        if keys[i]==idc:
            continue
        cur.execute('SELECT fingerprint from users where id=?',(keys[i],))
        fingerPrint2 = cur.fetchone()[0]
        fingerPrint1 = fingerPrint1.astype(np.float32)
        fingerPrint2 = fingerPrint2.astype(np.float32)
        C = getCorr(fingerPrint1,fingerPrint2)
        x,_ = getPCE(C)
        if x>max:
            max = x
            key = keys[i]
    return key

def highestPCE(fingerprint,imgs,ino,j):
    pceValues = np.zeros(j)
    for k in range(j):
        if k in ino:
            continue
        xt,pceCorr = getResults(fingerPrint,imgs[k])
        pceValues[k] = pceCorr
        if xt<0.85:
            verd = 'Rejected'
        else:
            verd = 'Accepted'
        # print('Image '+str(k)+' = '+str(pceCorr))
        #writer.writerows([[prevID,docsList[k],yt,xt,verd]])
    t = np.argmax(pceValues)
    if(pceValues[t]<=1):
        return fingerprint,ino
    ino.append(t)
    cluster_imgs = imgs[ino]
    return getFingerprintUtil(cluster_imgs),ino

def getKorus(fp,img):
    img = np.uint8(img)
    img_noise = noiseExtractFromImg(img,3)
    fp = fp * img
    fp = fp - np.mean(fp)
    img_noise = img_noise - np.mean(img_noise)
    n1 = np.sqrt(np.sum(img_noise*img_noise))
    n2 = np.sqrt(np.sum(fp*fp))
    return np.sum(img_noise*fp)/(n1*n2)

def getKorusFromPrnu(fp1,fp2):
    fp = fp1.copy()
    tt = fp2.copy()
    fp = fp * fp2
    fp = fp - np.mean(fp)
    tt = tt - np.mean(tt)
    n1 = np.sqrt(np.sum(tt*tt))
    n2 = np.sqrt(np.sum(fp*fp))
    return np.sum(tt*fp)/(n1*n2)

def getInitH(imgs):
    n = imgs.shape[0]
    fingerPrints = np.zeros(imgs.shape[:3])
    corrMat = np.zeros([n,n])
    for i in range(n):
        fingerPrints[i] = getFingerprintUtil(np.float32([imgs[i]]))
    for i in range(n):
        for j in range(n):
            if i is j:
                corrMat[i][j] = 0.0
                continue
            corrMat[i][j] = corr2(fingerPrints[i],fingerPrints[j])
    return corrMat

def getProjection(arr,random_arr):
    #proj = np.matmul(arr,random_arr)
    proj = arr
    proj = (proj>=0)*1
    proj = np.uint8(proj.flatten())
    return proj

In [11]:
# img_list = os.listdir('.')
# t = cv2.imread(img_list[0])
# t = (t-128)/128
# imgs = np.array([t])
# for i in img_list:
#     t = cv2.imread(i)
#     if t.shape[0]==imgs.shape[2] or t.shape[0]==imgs.shape[1]:
#         pass
#     else:
#         continue
#     if t.shape[0]==imgs.shape[2]:
#         t = imutils.rotate_bound(t,90)
#     t = (t-128)/128
#     imgs = np.append(imgs,[t],axis=0)
# print(imgs.shape)

img_list = os.listdir('.')
t = cv2.imread(img_list[0])
imgs = np.array([getFingerprintUtil(np.array([t]))])
for i in img_list:
    t = cv2.imread(i)
    if t.shape[0]==imgs.shape[2] or t.shape[0]==imgs.shape[1]:
        pass
    else:
        continue
    if t.shape[0]==imgs.shape[2]:
        t = imutils.rotate_bound(t,90)
    t = getFingerprintUtil(np.array([t]))
    imgs = np.append(imgs,[t],axis=0)
print(imgs.shape)

(370, 256, 256, 3)


In [12]:
cd /home/ubuntu/PRNU/data/prnu/

/home/ubuntu/PRNU/data/prnu


In [13]:
prnu_list = os.listdir('.')
prnus = np.array([np.load(prnu_list[0])])
for i in prnu_list[1:]:
    t = np.load(i)
    prnus = np.append(prnus,[t],axis=0)
print(prnus.shape)

(280, 256, 256, 3)


In [14]:
cd /home/ubuntu/PRNU/data/

/home/ubuntu/PRNU/data


In [15]:
from keras.optimizers import Adam

g = generator_model()
d = discriminator_model()
d_on_g = generator_containing_discriminator_multiple_outputs(g, d)

g_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
d_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
d_on_g_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

d.trainable = True
d.compile(optimizer=d_opt, loss=wasserstein_loss)
d.trainable = False
loss = [perceptual_loss, wasserstein_loss]
loss_weights = [100, 1]
d_on_g.compile(optimizer=d_on_g_opt, loss=loss, loss_weights=loss_weights)
d.trainable = True

In [16]:
def save_all_weights(d, g, epoch_number, current_loss):
    now = datetime.datetime.now()
    save_dir = os.path.join(BASE_DIR, '{}{}'.format(now.month, now.day))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    g.save_weights(os.path.join(save_dir, 'generator_{}_{}.h5'.format(epoch_number, current_loss)), True)
    d.save_weights(os.path.join(save_dir, 'discriminator_{}.h5'.format(epoch_number)), True)

In [17]:
epoch_num = 10
batch_size = 8
critic_updates = 5
BASE_DIR = 'weights/'
x_train = imgs.copy()[:280]
y_train = prnus.copy()
output_true_batch, output_false_batch = np.ones((batch_size, 1)), np.zeros((batch_size, 1))

for epoch in range(epoch_num):
    print('epoch: {}/{}'.format(epoch, epoch_num))
    print('batches: {}'.format(x_train.shape[0] / batch_size))
    
    permutated_indexes = np.random.permutation(x_train.shape[0])
    
    d_losses = []
    d_on_g_losses = []
    
    for index in range(int(x_train.shape[0] / batch_size)):
        batch_indexes = permutated_indexes[index*batch_size:(index+1)*batch_size]
        image_blur_batch = x_train[batch_indexes]
        image_full_batch = y_train[batch_indexes]
        
        generated_images = g.predict(x=image_blur_batch, batch_size=batch_size)
        
        for _ in range(critic_updates):
            d_loss_real = d.train_on_batch(image_full_batch, output_true_batch)
            d_loss_fake = d.train_on_batch(generated_images, output_false_batch)
            d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
            d_losses.append(d_loss)
        print('batch {} d_loss : {}'.format(index+1, np.mean(d_losses)))
        
        d.trainable = False
        
        d_on_g_loss = d_on_g.train_on_batch(image_blur_batch, [image_full_batch, output_true_batch])
        d_on_g_losses.append(d_on_g_loss)
        print('batch {} d_on_g_loss : {}'.format(index+1, d_on_g_loss))
        
        d.trainable = True
        
    with open('log.txt', 'a') as f:
        f.write('{} - {} - {}\n'.format(epoch, np.mean(d_losses), np.mean(d_on_g_losses)))
    
    save_all_weights(d, g, epoch, int(np.mean(d_on_g_losses)))

epoch: 0/10
batches: 35.0
batch 1 d_loss : 0.056820625916589054
batch 1 d_on_g_loss : [4326.8281, 43.264885, 0.33999485]
batch 2 d_loss : 0.04443447075318545
batch 2 d_on_g_loss : [4345.5825, 43.454971, 0.085366644]
batch 3 d_loss : 0.03371583658202629
batch 3 d_on_g_loss : [4195.7012, 41.956921, 0.0093528125]
batch 4 d_loss : 0.02550871691237262
batch 4 d_on_g_loss : [4659.2319, 46.592308, 0.0010895543]
batch 5 d_loss : 0.020466888699156697
batch 5 d_on_g_loss : [4653.4707, 46.534702, 0.0003546433]
batch 6 d_loss : 0.017083890238670088
batch 6 d_on_g_loss : [3572.7346, 35.727345, 0.00024797532]
batch 7 d_loss : 0.014663070048637955
batch 7 d_on_g_loss : [4171.8755, 41.71875, 0.00025255434]
batch 8 d_loss : 0.012841863626090345
batch 8 d_on_g_loss : [3723.8533, 37.238529, 0.00023604796]
batch 9 d_loss : 0.011424010745637739
batch 9 d_on_g_loss : [4176.085, 41.760845, 0.00025784786]
batch 10 d_loss : 0.010286838801403065
batch 10 d_on_g_loss : [3791.8276, 37.918274, 0.00014705356]
batch

batch 13 d_loss : 4.381472437951463e-06
batch 13 d_on_g_loss : [2868.7756, 28.687757, 8.5678294e-06]
batch 14 d_loss : 4.3101171221288884e-06
batch 14 d_on_g_loss : [3063.4949, 30.634949, 9.5370624e-06]
batch 15 d_loss : 4.280794685958729e-06
batch 15 d_on_g_loss : [3109.1353, 31.091352, 2.1156819e-05]
batch 16 d_loss : 4.341512180872087e-06
batch 16 d_on_g_loss : [3072.4497, 30.724497, 7.8034327e-06]
batch 17 d_loss : 4.253151452276677e-06
batch 17 d_on_g_loss : [3040.6099, 30.406099, 1.15432e-05]
batch 18 d_loss : 4.2279120786285296e-06
batch 18 d_on_g_loss : [2951.9702, 29.519703, 9.6877438e-06]
batch 19 d_loss : 4.115730433496585e-06
batch 19 d_on_g_loss : [2815.2961, 28.152962, 6.7446063e-06]
batch 20 d_loss : 4.036749526221684e-06
batch 20 d_on_g_loss : [3432.5479, 34.325478, 9.3345243e-06]
batch 21 d_loss : 4.005484981375741e-06
batch 21 d_on_g_loss : [3249.585, 32.49585, 8.3068262e-06]
batch 22 d_loss : 3.9432437502413945e-06
batch 22 d_on_g_loss : [3433.5979, 34.335979, 6.9992

batch 24 d_loss : 1.3416776918499333e-06
batch 24 d_on_g_loss : [2768.0981, 27.680981, 2.7704455e-06]
batch 25 d_loss : 1.3378949888647185e-06
batch 25 d_on_g_loss : [2875.5847, 28.755848, 3.7821424e-06]
batch 26 d_loss : 1.3200096807745117e-06
batch 26 d_on_g_loss : [2971.7922, 29.717922, 3.3166834e-06]
batch 27 d_loss : 1.3155814355269723e-06
batch 27 d_on_g_loss : [3005.7949, 30.057949, 2.192231e-06]
batch 28 d_loss : 1.3133668626811154e-06
batch 28 d_on_g_loss : [2896.3669, 28.963669, 2.7965e-06]
batch 29 d_loss : 1.339852336702185e-06
batch 29 d_on_g_loss : [2762.3943, 27.623943, 3.639605e-06]
batch 30 d_loss : 1.3421199760917564e-06
batch 30 d_on_g_loss : [2712.3782, 27.123781, 1.8530743e-06]
batch 31 d_loss : 1.3291483480403962e-06
batch 31 d_on_g_loss : [3107.9724, 31.079725, 4.2818292e-06]
batch 32 d_loss : 1.3118900088926466e-06
batch 32 d_on_g_loss : [2771.7603, 27.717602, 2.1626131e-06]
batch 33 d_loss : 1.3058112889060188e-06
batch 33 d_on_g_loss : [2744.2188, 27.442188, 4

batch 35 d_on_g_loss : [2548.4009, 25.484009, 1.4361697e-06]
epoch: 7/10
batches: 35.0
batch 1 d_loss : 6.402393637472414e-07
batch 1 d_on_g_loss : [2746.5938, 27.465937, 1.2454444e-06]
batch 2 d_loss : 5.591428902107509e-07
batch 2 d_on_g_loss : [2566.1138, 25.661139, 9.2063488e-07]
batch 3 d_loss : 5.733944268134413e-07
batch 3 d_on_g_loss : [2445.584, 24.455841, 1.3316723e-06]
batch 4 d_loss : 5.004842350331274e-07
batch 4 d_on_g_loss : [2851.4417, 28.514416, 1.3495935e-06]
batch 5 d_loss : 5.012899748635391e-07
batch 5 d_on_g_loss : [2737.7952, 27.377951, 2.7917395e-06]
batch 6 d_loss : 5.23509784026525e-07
batch 6 d_on_g_loss : [3145.9734, 31.459734, 1.5841185e-06]
batch 7 d_loss : 5.262112990424482e-07
batch 7 d_on_g_loss : [2522.2554, 25.222553, 1.2201297e-06]
batch 8 d_loss : 5.194707583200397e-07
batch 8 d_on_g_loss : [2792.1902, 27.921902, 1.9913105e-06]
batch 9 d_loss : 5.285733373562431e-07
batch 9 d_on_g_loss : [2547.928, 25.47928, 1.815177e-06]
batch 10 d_loss : 5.5214956

batch 12 d_loss : 3.205422961135203e-07
batch 12 d_on_g_loss : [2614.3018, 26.143019, 9.9537635e-07]
batch 13 d_loss : 3.192425157731822e-07
batch 13 d_on_g_loss : [2648.9675, 26.489676, 5.8862906e-07]
batch 14 d_loss : 3.188771513253284e-07
batch 14 d_on_g_loss : [2476.0574, 24.760574, 1.1602351e-06]
batch 15 d_loss : 3.3393117215988847e-07
batch 15 d_on_g_loss : [2942.1365, 29.421366, 8.4316173e-07]
batch 16 d_loss : 3.268904279707385e-07
batch 16 d_on_g_loss : [2411.7854, 24.117855, 9.6563849e-07]
batch 17 d_loss : 3.1672417435402165e-07
batch 17 d_on_g_loss : [2626.6196, 26.266195, 7.6030057e-07]
batch 18 d_loss : 3.116777373672145e-07
batch 18 d_on_g_loss : [2917.6152, 29.176153, 1.1180941e-06]
batch 19 d_loss : 3.0735210998500406e-07
batch 19 d_on_g_loss : [2610.1965, 26.101965, 8.1187426e-07]
batch 20 d_loss : 3.105986317564202e-07
batch 20 d_on_g_loss : [2623.8457, 26.238457, 1.2112848e-06]
batch 21 d_loss : 3.087102037452063e-07
batch 21 d_on_g_loss : [2639.03, 26.390301, 1.01

In [18]:
cd /home/ubuntu/PRNU/data/imgs/

/home/ubuntu/PRNU/data/imgs


In [19]:
x_tt = cv2.imread('Kodak_M1063_34.jpg')
x_tt = getFingerprintUtil(np.array([x_tt]))
x_tt = np.array([x_tt])
x_tt.shape

(1, 256, 256, 3)

In [20]:
cd ../weights/713/

/home/ubuntu/PRNU/data/weights/713


In [21]:
g = generator_model()
g.load_weights('generator_9_896.h5')
generated_images = g.predict(x=x_tt)

In [22]:
np.save('a.npy',generated_images[0])

In [40]:
cv2.imwrite('aa.jpg',generated_images[0]*255)

True